### Experimento de procesamiento por lotes en GPU con `cudf` y `cupy`
Este experimento busca realizar una prueba de rendimiento en la GPU utilizando `cudf` para el manejo de DataFrames y `cupy` para operaciones numéricas de gran escala. La intención es verificar la capacidad de la GPU para manejar y procesar grandes cantidades de datos mediante un enfoque de procesamiento por lotes, liberando la memoria después de cada lote para evitar saturaciones de memoria.

**Configuración del experimento**
* Total de Filas: 1,000,000,000
* Tamaño del Lote: 100,000

**Objetivos del experimento**
1. Generar datos sintéticos que simulen un gran volumen de datos en GPU para operaciones de análisis.
2. Ejecutar procesamiento por lotes para evitar el uso excesivo de memoria GPU, al dividir los datos en bloques más pequeños.
3. Optimizar el uso de memoria en GPU mediante la liberación de memoria después de cada lote.
4. Medir el tiempo total de procesamiento y el uso de memoria de la GPU durante la ejecución.

In [1]:
!nvidia-smi

Tue Nov 12 18:30:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   45C    P5             25W /  285W |    1504MiB /  12282MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import cudf as cd
import cupy as cp
from time import time
from tqdm import tqdm
from IPython.display import HTML

In [3]:
# Parámetros de los experimentos

# Tamaño de las matrices y el tamaño del batch
num_rows = 1_000_000_000
batch_size = 100_000

# Definir el límite máximo de VRAM en uso antes de liberar memoria (95% de VRAM total)
_, vram_total = cp.cuda.runtime.memGetInfo()
vram_limit = vram_total * 0.95

# Definir el número de batches en cada iteración y almacenamiento de resultados
num_batches = num_rows // batch_size
results = {
    'exp1': list(),
    'exp2': list()
}

In [4]:
# Generación de un DataFrame sintético grande
def generate_data(rows):
    data = {
        'id': cp.random.randint(1, 1_000_000, size=rows),
        'value1': cp.random.rand(rows) * 100,
        'value2': cp.random.rand(rows) * 100,
        'category': cp.random.randint(0, 4, size=rows)  # Genera categorías numéricas
    }
    # Opcional: mapea los números a letras después si es necesario
    df = cd.DataFrame(data)
    df['category'] = df['category'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D'})
    return df

# Procesamiento por lotes en la GPU
def batch_processing(df):
    # Agrupación por 'category' y cálculo de media y suma
    grouped = df.groupby('category').agg({
        'value1': ['mean', 'sum'],
        'value2': ['mean', 'sum']
    })
    return grouped

In [5]:
print('Experimento 1\nIniciando procesamiento por lotes...')
start_time = time()

# Añadir tqdm a las iteraciones para ver el progreso
for i in tqdm(range(num_batches), desc='Procesando lotes'):
    # Genera un lote de datos y pasa a cudf DataFrame
    batch_df = generate_data(batch_size)
    # Procesamiento del lote
    batch_result = batch_processing(batch_df)
    results['exp2'].append(batch_result)
    
    # Verifica el uso de memoria y libera si se excede el límite
    vram_free, vram_total = cp.cuda.runtime.memGetInfo()
    vram_used = vram_total - vram_free
    if vram_used >= vram_limit:
        cp.get_default_memory_pool().free_all_blocks()

print(f'Procesamiento completo en {time() - start_time:.2f} segundos.')
# Chequeo de uso de memoria en GPU final
vram_free, vram_total = cp.cuda.runtime.memGetInfo()
vram_used = vram_total - vram_free
print(f'Uso de memoria en GPU: {vram_used / (1024**3):.2f} GB usado de {vram_total / (1024**3):.2f} GB total.')

Experimento 1
Iniciando procesamiento por lotes...


Procesando lotes: 100%|██████████| 10000/10000 [01:37<00:00, 102.07it/s]

Procesamiento completo en 97.99 segundos.
Uso de memoria en GPU: 1.26 GB usado de 11.99 GB total.


In [ ]:
# Especifica la ruta al archivo de video
HTML("""
<video width="600" height="400" controls>
  <source src="./exp1.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [7]:
print('Experimento 2\nIniciando procesamiento por lotes...')
start_time = time()

# Añadir tqdm a las iteraciones para ver el progreso
for i in tqdm(range(num_batches), desc='Procesando lotes'):
    # Genera un lote de datos y pasa a cudf DataFrame
    batch_df = generate_data(batch_size)
    # Procesamiento del lote
    batch_result = batch_processing(batch_df)
    # Guardamos el resultado para evitar que sea eliminado de la memoria
    results['exp2'].append(batch_result)
    # Forzamos que cupy libere memoria no utilizada
    cp.get_default_memory_pool().free_all_blocks()

print(f'Procesamiento completo en {time() - start_time:.2f} segundos.')

# Chequeo de uso de memoria en GPU final
vram_free, vram_total = cp.cuda.runtime.memGetInfo()
vram_used = vram_total - vram_free
print(f'Uso de memoria en GPU: {vram_used / (1024**3):.2f} GB usado de {vram_total / (1024**3):.2f} GB total.')

Experimento 2
Iniciando procesamiento por lotes...


Procesando lotes: 100%|██████████| 10000/10000 [01:14<00:00, 133.96it/s]

Procesamiento completo en 74.65 segundos.
Uso de memoria en GPU: 1.29 GB usado de 11.99 GB total.


In [8]:
# Especifica la ruta al archivo de video
HTML("""
<video width="600" height="400" controls>
  <source src="./exp2.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")